<a href="https://colab.research.google.com/github/ArunGNair06/123/blob/main/IMDLIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Connect Google drive with Google colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install imdlib # Cell 2: install dependencies (run once)
pip install imdlib geopandas rioxarray xarray rasterio pyproj shapely
# rioxarray installs rasterio/xarray and makes reprojection easier


In [ ]:
# Cell 3: main script - download IMD data, clip to AOI, export CSV
import os
import warnings
warnings.filterwarnings("ignore")

import imdlib as imd
import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# ----------------- USER CONFIG -----------------
# path on your Google Drive where data will be saved / read
drive_base = "/drive/MyDrive/GEE_Export/Rainfall"

# IMD time range & variable
start_yr = 2019
end_yr = 2020
variable = "rain"   # 'rain', 'tmin', 'tmax' (as supported by imdlib)

# your AOI shapefile path (on Drive). It can be a single polygon or multi-polygon.
shapefile_path = "/drive/MyDrive/GEE_Exports_NDVI_LST/KochiCorp_shapefile.shp"

# output CSV path (combined)
out_csv = os.path.join(drive_base, f"imd_{variable}_{start_yr}_{end_yr}_points.csv")

# If you prefer separate CSV per polygon (if shapefile has multiple features), set True:
save_one_csv_per_polygon = False
one_csv_folder = os.path.join(drive_base, "per_polygon_csvs")
os.makedirs(one_csv_folder, exist_ok=True)

# ------------------------------------------------

os.makedirs(drive_base, exist_ok=True)

print("Downloading IMD data (this may take a while the first time)...")
# This will download yearwise files into drive_base (folder)
imd.get_data(variable, start_yr, end_yr, fn_format="yearwise", file_dir=drive_base)

print("Opening downloaded data...")
# open_data returns an object from imdlib; get_xarray gives xarray.Dataset
data_obj = imd.open_data(variable, start_yr, end_yr, "yearwise", drive_base)
ds = data_obj.get_xarray()   # xarray Dataset

# pick the main data variable inside xarray dataset
# usually ds[variable] exists; if not, pick the first data variable
if variable in ds.data_vars:
    da = ds[variable]
else:
    # fallback: pick first data variable
    first_var = list(ds.data_vars.keys())[0]
    print(f"Warning: variable '{variable}' not found in dataset. Using '{first_var}' instead.")
    da = ds[first_var]

print("Dataset dims:", ds.dims)
print("Data variable dims:", da.dims)
print(da)
# Ensure lat/lon dims exist; common names: 'lat' and 'lon' or 'latitude'/'longitude'
lat_name = None
lon_name = None
for name in ("lat", "latitude"):
    if name in da.coords:
        lat_name = name
        break
for name in ("lon", "longitude", "lonc"):
    if name in da.coords:
        lon_name = name
        break

if lat_name is None or lon_name is None:
    raise RuntimeError(f"Could not find lat/lon coordinate names. Coordinates available: {list(da.coords)}")

print("Using coordinates:", lat_name, lon_name)

# Convert DataArray into a long-form DataFrame [lon, lat, time, value]
# We stack the spatial dims into a single index then reset index to columns
# If time dimension exists, it will be preserved.

# Make sure coordinates are 1D and sorted (xarray usually has them)
da_stacked = da.stack(points=(lat_name, lon_name)).reset_index("points")
# The stack produced an index 'points' containing tuple (lat, lon) or indices; convert to coords
# Simpler: convert full dataset to dataframe then reset index
df = da.to_dataframe().reset_index()

# Columns typically include lat_name, lon_name, maybe 'time' and the data variable column (same as variable name or first_var)
# Normalize column names
value_col = list(da.coords.keys())[0] if False else (variable if variable in da.coords else da.name)
# Actually da.to_dataframe places the variable as a column with da.name
# Find actual column representing values:
possible_value_cols = [c for c in df.columns if c not in [lat_name, lon_name, "time", "year", "points"]]
# choose the column that is numeric and not coordinate
val_col = None
for c in possible_value_cols:
    if pd.api.types.is_numeric_dtype(df[c]):
        val_col = c
        break
if val_col is None:
    # fallback: the last column
    val_col = df.columns[-1]

print("Value column detected as:", val_col)

# If dataset uses 'year' instead of 'time', we keep whichever exists
time_col = "time" if "time" in df.columns else ("year" if "year" in df.columns else None)

# Create GeoDataFrame of grid points
gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df[lon_name], df[lat_name]),
    crs="EPSG:4326"   # IMD lat/lon are geographic
)

# Read AOI shapefile and reproject to match points CRS (EPSG:4326)
aoi = gpd.read_file(shapefile_path)
if aoi.crs is None:
    print("Warning: AOI shapefile has no CRS. Assuming EPSG:4326. If wrong, reproject your shapefile.")
    aoi = aoi.set_crs("EPSG:4326")
else:
    aoi = aoi.to_crs("EPSG:4326")

# Option: dissolve all polygons into one (if you want a single AOI)
# aoi_union = aoi.unary_union
# If you want per-polygon outputs, keep original features

# Spatial join: keep only grid points inside AOI polygons
print("Filtering points inside AOI...")
# We'll spatial join points to AOI, producing polygon attribute columns (if any)
joined = gpd.sjoin(gdf_points, aoi, how="inner", predicate="within")
# joined now contains columns from both; for multi-polygons each point gets the polygon attributes.

if joined.empty:
    print("No grid points found inside AOI. Check shapefile and dataset extents.")
else:
    # Keep relevant columns: time (if exists), lat, lon, value
    cols_to_keep = [lon_name, lat_name, val_col]
    if time_col:
        cols_to_keep.insert(0, time_col)
    # include AOI attributes for identification (like index_right or any ID)
    # 'index_right' is added by sjoin: it links to AOI row index
    if "index_right" in joined.columns:
        cols_to_keep.append("index_right")

    export_df = joined[cols_to_keep].copy()
    # rename columns to consistent names
    rename_map = {lon_name: "lon", lat_name: "lat", val_col: variable}
    if time_col:
        rename_map[time_col] = "time"
    export_df = export_df.rename(columns=rename_map)

    # Save combined CSV
    os.makedirs(os.path.dirname(out_csv) or ".", exist_ok=True)
    export_df.to_csv(out_csv, index=False)
    print("Saved combined CSV:", out_csv)

    # Optionally save one CSV per polygon (index_right)
    if save_one_csv_per_polygon:
        for idx, grp in export_df.groupby("index_right"):
            poly_csv = os.path.join(one_csv_folder, f"polygon_{idx}_{variable}_{start_yr}_{end_yr}.csv")
            grp.to_csv(poly_csv, index=False)
        print("Saved per-polygon CSVs to:", one_csv_folder)


In [ ]:
# Provide the alttitude & Longitude of a point for which the data is required
#  And save the data into CSV file

lat = 20.03 #lattitude of point
lon = 77.23 #longitude of point
data.to_csv('data.csv', lat, lon, path)

In [ ]:
# Save CSV files for multiple points

# Provide lat and long in a list
latLong = [[20.3,77.23],[23.5,72.5],[26.0,77,1]]

for points in latLong:
  lat = points[0]
  lon = points[1]

  data.to_csv('test.csv', lat, lon, path)
  print ("data save for ",points)

In [ ]:
#  Provide the Geojson file of a catchment or polygon to dowlnaod all the gridded data lying into that polygon

geojson_file = '/content/drive/MyDrive/IMD/Test_geojson.geojson'


url="https://drive.google.com/file/d/111XvmUzvTlhY2lbFMseGVhZQh4pisFXQ/view?usp=sharing"
url2='https://drive.google.com/uc?id=' + url.split('/')[-2]

points_df = pd.read_csv(url2)


geometry = [Point(xy) for xy in zip(points_df['Long'], points_df['Lat'])]

# Creating the GeoDataFrame
gdf_points = gpd.GeoDataFrame(points_df, geometry=geometry)

# Set a CRS (coordinate reference system), EPSG:4326 is WGS84 Lat/Long
gdf_points.set_crs(epsg=4326, inplace=True)


gdf_polygon = gpd.read_file(geojson_file)

# Ensure both GeoDataFrames use the same CRS
if gdf_points.crs != gdf_polygon.crs:
    gdf_points = gdf_points.to_crs(gdf_polygon.crs)

gdf_list = []
for row in range (len(gdf_polygon)):
    points_in_polygon = gdf_points[gdf_points.within(gdf_polygon.iloc[row].geometry)]
    gdf_list.append(points_in_polygon)

final_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

final_df = final_gdf[["Name","Lat","Long"]]
final_df.to_csv("Master_file.csv")

for index, row in final_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    data.to_csv('test.csv', lat, lon, path)
    print ("data save for " + str(lat)+ "_" +str(lon))